In [ ]:
# Lab 3: web scraping and data storage with requests, beautifulsoup, sqlite3, review tkinter
# Name: Minh Duc Vo
# Description: lab3back
import requests
!pip install lxml
from bs4 import BeautifulSoup 
import json
import sqlite3
import numpy as np

In [ ]:
def run():
    ROOT = "https://guide.michelin.com"
    d = {'URL':[], 'names':[], 'location':[], 'cost':[], 'cuisine':[],'address':[]}
    page_num = 0
    while True:
        if page_num:
            page = requests.get(ROOT+f'/us/en/california/san-jose/restaurants/page/{num+1}')
        else:
            page = requests.get(ROOT+'/us/en/california/cupertino/restaurants')
        page_num += 1
        soup = BeautifulSoup(page.content, "html.parser")
        if page.status_code == 404:
            break
        for tag in soup.find_all( 'div', class_= 'card__menu-content js-match-height-content' ):
                d['location'].append(tag.find('div', class_ = 'card__menu-footer--location flex-fill pl-text').text.rstrip().lstrip())
                cost, cuisine = tag.find('div', class_ = 'card__menu-footer--price pl-text').text.replace('\n','').replace(' ', '').split('·')
                d['cuisine'].append(cuisine)
                d['cost'].append(cost)
                d['URL'].append(tag.find('a')['href'])
                d['names'].append(tag.select('h3 a')[0].text.rstrip().lstrip())
        for url in d['URL'][(num*20):]:
            d['address'].append(BeautifulSoup(requests.get(ROOT + url).content, "html.parser").find('li', class_ = 'restaurant-details__heading--address').text)
        with open('lab3_cupertino.json', 'w') as fh:
            json.dump(d, fh)
        with open('lab3_cupertino.json', 'r') as fh:
            data = json.load(fh)
    val = {}
    db = sqlite3.connect('lab3_cupertino.db')
    cur = db.cursor()
    for key in np.array([*data.keys()])[2:5]:
        cur.execute(f"DROP TABLE IF EXISTS {str(key).capitalize()}")      
        cur.execute(f'''CREATE TABLE {str(key).capitalize()}(             
                           id INTEGER NOT NULL PRIMARY KEY,
                           {key} TEXT UNIQUE ON CONFLICT IGNORE)''')
    cur.execute("DROP TABLE IF EXISTS Lab3")      
    cur.execute('''CREATE TABLE Lab3(             
                       id INTEGER NOT NULL PRIMARY KEY UNIQUE,
                       url TEXT,
                       names TEXT,
                       location INTEGER,
                       cost INTEGER,
                       cuisine INTEGER,
                       address TEXT)''')

    for id in range(len(data['names'])) :
        for key in np.array([*data.keys()])[2:5]:
            cur.execute(f'''INSERT INTO {str(key).capitalize()} ({key}) VALUES ('{data[key][id]}')''')
            cur.execute(f'''SELECT id FROM {str(key).capitalize()} WHERE {key} = '{data[key][id]}' ''')
            val[key] = cur.fetchone()[0]
        #I thought I can unpack in (), but I cannot???
        cur.execute('''INSERT INTO Lab3
            (url, names, location, cost, cuisine, address) 
            VALUES ( ?, ?, ?, ?, ?, ? )''', (data['URL'][id], data['names'][id], val['location'], val['cost'], val['cuisine'], data['address'][id]) )
    db.commit()
    db.close()

In [ ]:
import sqlite3
import tkinter as tk
import tkinter.messagebox as tkmb
# Lab 3: web scraping and data storage with requests, beautifulsoup, sqlite3, review tkinter
# Name: Minh Duc Vo
# Description: lab3front
import webbrowser
import os.path

# from lab3back import run
# run()
class MainWin(tk.Tk):
    '''
        Main window class: Explain the application with 2 buttons for the users to choose (and all of the window calls)
    '''
    DB_PATH = 'lab3_cupertino.db'
    def __init__(self):
        super().__init__()
        self.title("Restaurants")
        self.geometry("230x80")
        try:
            if not os.path.isfile(MainWin.DB_PATH):
                raise FileNotFoundError('Database file not found')
            self.db = sqlite3.connect(MainWin.DB_PATH)
            self.cur = self.db.cursor()
            tk.Label(self, text= "Local Michelin Guide Restaurants", fg = "blue", font=("Calibri", 12, "") ).grid(row = 0, column = 0, columnspan = 5)
#             self.columnconfigure(7, minsize=50)
            tk.Button(self, text="City", fg = "black", command=lambda: self.Choice("location")).grid(row = 2, column = 1)
            tk.Button(self, text="Cuisine", fg = "black", command=lambda: self.Choice("cuisine")).grid(row = 2, column = 3)
            tk.Label(self, text= "Search by:", fg = "black").grid(row= 1, column = 2)
            self.protocol("WM_DELETE_WINDOW", self.closefct)
        except FileNotFoundError as e:
            self.errorfct(e)
            self.quit()
        
    def Choice(self, choice):
        """Action by choice"""
        w = DialogWin(self, choice, self.db, self.cur)
        w.grab_set()
        self.wait_window(w)
        value = w.get()
        if value:
            w = DialogWin(self, choice, self.db, self.cur, value)
            w.grab_set()
            self.wait_window(w)
            value = w.get()
            for restaurant in value:
                w = DisplayWin(self, restaurant, self.db, self.cur)

    def closefct(self) :
        """Print out the ask to cancel message"""
        if tkmb.askokcancel("Confirm close", "Are you sure you want to close?", parent=self) :
            self.db.close()
            self.destroy()
            self.quit()
    def errorfct(self, message):
        """Print out the error message"""
        tkmb.showerror("Error", message, parent=self)
        self.destroy()
        self.quit()      
class DialogWin(tk.Toplevel):
    '''
        Dialog window class: Display a listbox of cities or cuisines or restaurants.
    '''
    def __init__(self, master, choice, db, cur, findRes = None):
        super().__init__(master)
        self._data = None
        self.findRes = findRes
        tk.Label(self, text="Click on a city to select", fg = "blue", font=("Calibri", 15, "") ).grid(row = 0, column = 1, columnspan = 2, sticky = 'w')
        self.lb = tk.Listbox(self, height = 6, width = 35, selectmode = 'multiple', font=("Calibri", 15))
        self.lb.grid(row = 1, column = 0, columnspan = 3, sticky="w")
        sb = tk.Scrollbar(self)
        sb.grid(row=1, column=3, sticky="nse")
        self.lb.config(yscrollcommand=sb.set)
        sb.config(command=self.lb.yview)
        if not findRes:
            self.title(f"{choice}")
            for record in cur.execute(f"SELECT {choice}.{choice} FROM {choice}"):
                self.lb.insert(tk.END, record[0])
        else:
            self.title(f"restaurant: {choice}")
            for index in findRes:
                cur.execute(f'''SELECT Lab3.names FROM Lab3 WHERE Lab3.{choice} = {index + 1}''')
                self.lb.insert(tk.END, cur.fetchone()[0])
        tk.Button(self, text="Click to select", command=self.LockIn).grid(row = 2, column = 1, sticky = 'e')
    def LockIn(self):
        """Receive the choice and destroy the Toplevel window"""
        if self.findRes:
            self._data = [self.lb.get(restaurant) for restaurant in self.lb.curselection()]
        else:
            self._data = self.lb.curselection()
        self.destroy()
        
    def get(self):
        '''Returns data'''
        return self._data
        
        
class DisplayWin(tk.Toplevel):
    '''
        Display window class:
        > Display the restaurant name, address, cost, and cuisine are displayed, one per line.
        > The user can use the button to open a tab on their browser with the Michelin Guide page of the restaurant
    '''
    ROOT = "https://guide.michelin.com"
    def __init__(self, master, restaurant, db, cur):
        super().__init__(master)
        cur.execute(f'''SELECT Lab3.names, Lab3.address, cost.cost, cuisine.cuisine, Lab3.URL FROM Lab3 JOIN location JOIN cost JOIN cuisine
                        ON Lab3.location = location.id AND Lab3.cost = cost.id AND Lab3.cuisine = cuisine.id
                        WHERE Lab3.names LIKE '{restaurant.split(' ')[0]}%' ''')
        values = cur.fetchall()
        tk.Label(self, text= f'{values[0][0]}', fg = "blue", font=("Calibri", 15, "") ).grid(row = 0, columnspan = 3, sticky = "nsew")
        tk.Label(self, text= f'{values[0][1]}', font=("Calibri", 15, "") ).grid(row = 1, columnspan = 3, sticky = "nsew")
        tk.Label(self, text= f'Cost: {values[0][2]}', font=("Calibri", 15, "") ).grid(row = 2, columnspan = 3, sticky = "nsew")
        tk.Label(self, text= f'Cuisine : {values[0][3]}', font=("Calibri", 15, "") ).grid(row = 3, columnspan = 3, sticky = "nsew")
        tk.Button(self, text="Visit webpage", fg = "blue", command=lambda:webbrowser.open_new(DisplayWin.ROOT+values[0][4])).grid(row = 4, column = 1,  sticky = "nsew")
app = MainWin()
app.mainloop()